# はじめに

このチュートリアルでは、Azure Cosmos DB for MongoDB vCoreに格納されたサンプルデータセットを活用して、OpenAIモデルを基盤にする方法を示します。これにより、Azure Cosmos DB for MongoDB vCoreの[ベクトル類似検索](https://learn.microsoft.com/azure/cosmos-db/mongodb/vcore/vector-search) 機能を活用します。 最終的には、データセットに基づいてAzureサービスに関する質問に答えるために、GPT-3.5の補完モデルを用いたインタラクティブなチャットセッションを作成します。このプロセスは、Retrieval Augmented Generation（RAG）として知られています。

このチュートリアルでは、Azure Cognitive Search Vector Searchデモからいくつかのコードスニペットとサンプルデータを借用しています。

# 事前準備 <a class="anchor" id="preliminaries"></a>
まずは、後で必要になるパッケージをインストールしてみましょう。

In [ ]:
! pip install numpy
! pip install openai
! pip install pymongo
! pip install python-dotenv
! pip install azure-core
! pip install azure-cosmos

In [ ]:
import json
import datetime
import time
import urllib 

from azure.core.exceptions import AzureError
from azure.core.credentials import AzureKeyCredential
import pymongo

from openai import AzureOpenAI
from dotenv import load_dotenv

example.envファイルをテンプレートとして使用して、独自の.envファイルに必要なキーとエンドポイントを提供してください。env_nameを適切に変更してください。

In [ ]:
from dotenv import dotenv_values

# specify the name of the .env file name 
env_name = "example.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)

COSMOS_MONGO_USER = config['cosmos_db_mongo_user']
COSMOS_MONGO_PWD = config['cosmos_db_mongo_pwd']
COSMOS_MONGO_SERVER = config['cosmos_db_mongo_server']

AOAI_client = AzureOpenAI(api_key=config['openai_api_key'], azure_endpoint=config['openai_api_endpoint'], api_version=config['openai_api_version'],)

## Azure Cosmos DB for MongoDB vCoreリソースの作成<a class="anchor" id="cosmosdb"></a>
まず、このクイックスタートガイドに従ってAzure Cosmos DB for MongoDB vCoreリソースを作成します: https://learn.microsoft.com/en-us/azure/cosmos-db/mongodb/vcore/quickstart-portal

次に、接続の詳細（サーバー、ユーザー、パスワード）をconfig.jsonファイルにコピーします。

## Azure OpenAI <a class="anchor" id="azureopenai"></a>

最後に、Azure OpenAIリソースをセットアップしましょう。現時点では、このサービスへのアクセスはアプリケーションによってのみ許可されています。Azure OpenAIへのアクセスを申請するには、[こちらのフォーム](https://aka.ms/oai/access)を完成させてください。アクセスを取得したら、以下の手順を完了してください:

- [クイックスタート](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource?pivots=web-portal)に従ってAzure OpenAIリソースを作成します。
- `completions`および`embeddings`モデルをデプロイします
    - `completions`に関する詳細は[こちら](https://learn.microsoft.com/azure/ai-services/openai/how-to/completions)を参照してください。
    - `embeddings`に関する詳細は[こちら](https://learn.microsoft.com/azure/ai-services/openai/how-to/embeddings)を参照してください。
- エンドポイント、キー、デプロイメント名（埋め込みモデル、補完モデル）をconfig.jsonファイルにコピーします。


# データをロードして埋め込みを作成する <a class="anchor" id="loaddata"></a>
ここでは、Azureサービスの説明を含むサンプルデータセットをロードし、そのデータからAzure OpenAIを使用してベクトル埋め込みを作成します。

In [ ]:
# Load text-sample_w_embeddings.json which has embeddings pre-computed
data_file = open(file="../../DataSet/AzureServices/text-sample_w_embeddings.json", mode="r") 

# OR Load text-sample.json data file. Embeddings will need to be generated using the function below.
# data_file = open(file="../../DataSet/AzureServices/text-sample.json", mode="r")

data = json.load(data_file)
data_file.close()

In [ ]:
# Take a peek at one data item
print(json.dumps(data[0], indent=2))

In [ ]:
def generate_embeddings(text):
    '''
    Generate embeddings from string of text.
    This will be used to vectorize data and user input for interactions with Azure OpenAI.
    '''
    response = AOAI_client.embeddings.create(input=text, model=config['openai_embeddings_deployment'])
    embeddings =response.model_dump()
    time.sleep(0.5) 
    return embeddings['data'][0]['embedding']

In [ ]:
# Generate embeddings for title and content fields
n = 0
for item in data:
    n+=1
    title = item['title']
    content = item['content']
    title_embeddings = generate_embeddings(title)
    content_embeddings = generate_embeddings(content)
    item['titleVector'] = title_embeddings
    item['contentVector'] = content_embeddings
    item['@search.action'] = 'upload'
    print("Creating embeddings for item:", n, "/" ,len(data), end='\r')
# Save embeddings to sample_text_w_embeddings.json file
with open("../../DataSet/AzureServices/text-sample_w_embeddings.json", "w") as f:
    json.dump(data, f)

In [ ]:
# Take a peek at one data item with embeddings created
print(json.dumps(data[0], indent=2))

# Connect and setup Cosmos DB for MongoDB vCore

## 接続のセットアップ

In [ ]:
mongo_conn = "mongodb+srv://"+urllib.parse.quote(COSMOS_MONGO_USER)+":"+urllib.parse.quote(COSMOS_MONGO_PWD)+"@"+COSMOS_MONGO_SERVER+"?tls=true&authMechanism=SCRAM-SHA-256&retrywrites=false&maxIdleTimeMS=120000"
mongo_client = pymongo.MongoClient(mongo_conn)

## データベースとコレクションのセットアップ

In [ ]:
# create a database called TutorialDB
db = mongo_client['ExampleDB']

# Create collection if it doesn't exist
COLLECTION_NAME = "ExampleCollection"

collection = db[COLLECTION_NAME]

if COLLECTION_NAME not in db.list_collection_names():
    # Creates a unsharded collection that uses the DBs shared throughput
    db.create_collection(COLLECTION_NAME)
    print("Created collection '{}'.\n".format(COLLECTION_NAME))
else:
    print("Using collection: '{}'.\n".format(COLLECTION_NAME))

In [ ]:
# Use only if re-reunning code and want to reset db and collection
collection.drop_indexes()
mongo_client.drop_database("ExampleDB")

## ベクトルインデックスの作成

**重要: 1つのベクトルプロパティに対して1つのインデックスしか作成できません。** つまり、同じベクトルプロパティを指す複数のインデックスを作成することはできません。 インデックスのタイプを変更したい場合（たとえば、IVFからHNSWに）、新しいインデックスを作成する前に最初にインデックスを削除する必要があります。

### IVFインデックス
IVFは、クラスタリングを使用してデータセット内の類似したベクトルを高速に検索するための近似最近傍（ANN）アプローチです。 プロトタイプや小規模なデータセット（数千のドキュメント未満）の場合には良い選択肢です。 ただし、大規模なスケーリングやより高いスループットが必要な場合にはお勧めしません。

IVFは、無料のティアを含むすべてのクラスターティアでサポートされています。

In [ ]:
db.command({
  'createIndexes': 'ExampleCollection',
  'indexes': [
    {
      'name': 'VectorSearchIndex',
      'key': {
        "contentVector": "cosmosSearch"
      },
      'cosmosSearchOptions': {
        'kind': 'vector-ivf',
        'numLists': 1,
        'similarity': 'COS',
        'dimensions': 1536
      }
    }
  ]
})

### HNSWインデックス

HNSWは、Hierarchical Navigable Small Worldの略であり、グラフベースのインデックスで、ベクトルをクラスターとサブクラスターに分割します。 HNSWを使用すると、より高速でより正確な近似最近傍検索を行うことができます。 HNSWは、M40およびそれ以上のクラスターティアで利用可能です。

In [ ]:
db.command({ 
    "createIndexes": "ExampleCollection",
    "indexes": [
        {
            "name": "VectorSearchIndex",
            "key": {
                "contentVector": "cosmosSearch"
            },
            "cosmosSearchOptions": { 
                "kind": "vector-hnsw", 
                "m": 16, # default value 
                "efConstruction": 64, # default value 
                "similarity": "COS", 
                "dimensions": 1536
            } 
        } 
    ] 
}
)

## コレクションにデータをアップロードする
新しく作成したデータベースとコレクションに、JSON形式のデータを`insert_many()`を使用して簡単に挿入します。

In [ ]:
collection.insert_many(data)

# Vector Search in Cosmos DB for MongoDB vCore

In [ ]:
# Simple function to assist with vector search
def vector_search(query, num_results=5):
    query_embedding = generate_embeddings(query)
    embeddings_list = []
    pipeline = [
        {
            '$search': {
                "cosmosSearch": {
                    "vector": query_embedding,
                    "path": "contentVector",
                    "k": num_results#, #, "efsearch": 40 # optional for HNSW only 
                    #"filter": {"title": {"$ne": "Azure Cosmos DB"}}
                },
                "returnStoredSource": True }},
        {'$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' } }
    ]
    results = collection.aggregate(pipeline)
    return results

以下でテストクエリを実行しましょう。

In [ ]:
query = "What are some NoSQL databases in Azure?"#"What are the services for running ML models?"
results = vector_search(query)
for result in results: 
#     print(result)
    print(f"Similarity Score: {result['similarityScore']}")  
    print(f"Title: {result['document']['title']}")  
    print(f"Content: {result['document']['content']}")  
    print(f"Category: {result['document']['category']}\n")  

## フィルター付きベクトル検索（プレビュー）
ベクトル検索に追加のクエリフィルタを追加するには、フィルタ付きインデックスを作成し、検索パイプラインでそれを指定します。

**注意:** フィルター付きベクトル検索はプレビューであり、サブスクリプションでAzureプレビュー機能を有効にする必要があります。 プレビュー機能「filtering on vector search」を検索して有効にします。詳細については、[こちら](https://learn.microsoft.com/azure/azure-resource-manager/management/preview-features?tabs=azure-portal)を参照してください。

In [ ]:
# Add a filter index
db.command( {
    "createIndexes": "ExampleCollection",
    "indexes": [ {
        "key": { 
            "title": 1 
               }, 
        "name": "title_filter" 
    }
    ] 
} 
)

In [ ]:
# Verify all indexes are present
for i in collection.list_indexes():
    print(i)

In [ ]:
# Simple function to assist with vector search
def filtered_vector_search(query, num_results=5):
    query_embedding = generate_embeddings(query)
    embeddings_list = []
    pipeline = [
        {
            '$search': {
                "cosmosSearch": {
                    "vector": query_embedding,
                    "path": "contentVector",
                    "k": num_results,
                    "filter": {"title": {"$nin": ["Azure SQL Database", "Azure Database for MySQL"]}}
                },
                "returnStoredSource": True }},
        {'$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' } }
    ]
    results = collection.aggregate(pipeline)
    return results

In [ ]:
query = "What are some NoSQL databases in Azure?"#"What are the services for running ML models?"
results = filtered_vector_search(query)
for result in results: 
#     print(result)
    print(f"Similarity Score: {result['similarityScore']}")  
    print(f"Title: {result['document']['title']}")  
    print(f"Content: {result['document']['content']}")  
    print(f"Category: {result['document']['category']}\n")  

# データを使用した質疑応答（Q&A） with GPT-3.5

最後に、`Completions`モデルにプロンプトを供給するためのヘルパー関数を作成します。その後、モデルに質問を投げてデータに基づいた情報を受け取ることができるインタラクティブなループを作成します。

In [ ]:
#This function helps to ground the model with prompts and system instructions.

def generate_completion(vector_search_results, user_prompt):
    system_prompt = '''
    You are an intelligent assistant for Microsoft Azure services.
    You are designed to provide helpful answers to user questions about Azure services given the information about to be provided.
        - Only answer questions related to the information provided below, provide at least 3 clear suggestions in a list format.
        - Write two lines of whitespace between each answer in the list.
        - If you're unsure of an answer, you can say ""I don't know"" or ""I'm not sure"" and recommend users search themselves."
        - Only provide answers that have products that are part of Microsoft Azure and part of these following prompts.
    '''

    messages=[{"role": "system", "content": system_prompt}]
    for item in vector_search_results:
        messages.append({"role": "system", "content": item['document']['content']})
    messages.append({"role": "user", "content": user_prompt})
    response = AOAI_client.chat.completions.create(model=config['openai_completions_deployment'], messages=messages,temperature=0)
    
    return response.dict()

In [ ]:
# Create a loop of user input and model output. You can now perform Q&A over the sample data!

user_input = ""
print("*** Please ask your model questions about Azure services. Type 'end' to end the session.\n")
user_input = input("User prompt: ")
while user_input.lower() != "end":
    search_results = vector_search(user_input)
    completions_results = generate_completion(search_results, user_input)
    print("\n")
    print(completions_results['choices'][0]['message']['content'])
    user_input = input("User prompt: ")